## Pre-requisites

In [1]:
# Import libraries
import os
import json
import pandas as pd
import pyTigerGraph as tg
from dotenv import load_dotenv, find_dotenv
# load .env
load_dotenv(find_dotenv())

True

Ensure that the following credentials are replaced to match your solution!

In [2]:
TG_HOST = os.getenv("TG_HOST") # The link to GraphStudio
TG_USERNAME = os.getenv("TG_USERNAME") # The username
TG_PASSWORD = os.getenv("TG_PASSWORD") # The password
TG_GRAPHNAME = os.getenv("TG_GRAPHNAME") # The name of the graph

In [3]:
print(TG_HOST)

https://987c3d4375c043f5bf3691f630fdd055.i.tgcloud.io


In [45]:
# Establish a connection
conn = tg.TigerGraphConnection(host=TG_HOST, username=TG_USERNAME, password=TG_PASSWORD)

## Schema (Vertices, Edges, and Graph)

Define definition of vertices and edges

In [5]:
# Define vertices
query_create_vertices = """
  CREATE VERTEX Client (PRIMARY_ID id INT, birth_number INT) WITH primary_id_as_attribute="true"
  CREATE VERTEX Account (PRIMARY_ID id INT, frequency STRING, created_date INT) WITH primary_id_as_attribute="true"
  CREATE VERTEX District (PRIMARY_ID id INT, name STRING, region STRING) WITH primary_id_as_attribute="true"
  CREATE VERTEX CreditCard (PRIMARY_ID id INT, credit_type STRING, issued_number INT) WITH primary_id_as_attribute="true"
  CREATE VERTEX Loan (PRIMARY_ID id INT, amount FLOAT, duration INT, payments FLOAT, status STRING, created_date INT) WITH primary_id_as_attribute="true"
  CREATE VERTEX PermanentOrder (PRIMARY_ID id INT, bank_to STRING, account_to INT, amount FLOAT, k_symbol STRING) WITH primary_id_as_attribute="true"
  CREATE VERTEX Transaction (PRIMARY_ID id INT, created_date INT, trans_type STRING, operation STRING, amount FLOAT, balance FLOAT) WITH primary_id_as_attribute="true"
"""

# Define edges
query_create_edges = """
  CREATE UNDIRECTED EDGE has_account (FROM Client, TO Account)
  CREATE DIRECTED EDGE resides_in (FROM Client, TO District) WITH REVERSE_EDGE="reverse_resides_in"
  CREATE DIRECTED EDGE located_in (FROM Account, TO District) WITH REVERSE_EDGE="reverse_located_in"
  CREATE DIRECTED EDGE associated_with (FROM Account, TO Loan) WITH REVERSE_EDGE="reverse_associated_with"
  CREATE DIRECTED EDGE issued_card (FROM Account, TO CreditCard) WITH REVERSE_EDGE="reverse_issued_card"
  CREATE DIRECTED EDGE ordered_from (FROM Account, TO PermanentOrder) WITH REVERSE_EDGE="reverse_ordered_from"
  CREATE DIRECTED EDGE transacted_on (FROM Account, TO Transaction) WITH REVERSE_EDGE="reverse_transacted_on"
"""

In [10]:
# Define Graph with Specific Name
graph_name = "CzechGraph"
query_create_graph = """
  CREATE GRAPH %s (Client, Account, District, CreditCard, Loan, PermanentOrder, Transaction, has_account, resides_in, located_in, associated_with, issued_card, ordered_from, transacted_on)
""" % graph_name

Run the Queries: Create Vertices, Edges, and Graph

In [7]:
conn.gsql(query_create_vertices)

'Successfully created vertex types: [Client].\nSuccessfully created vertex types: [Account].\nSuccessfully created vertex types: [District].\nSuccessfully created vertex types: [CreditCard].\nSuccessfully created vertex types: [Loan].\nSuccessfully created vertex types: [PermanentOrder].\nSuccessfully created vertex types: [Transaction].'

In [8]:
conn.gsql(query_create_edges)

'Successfully created edge types: [has_account].\nSuccessfully created edge types: [resides_in].\nSuccessfully created reverse edge types: [reverse_resides_in].\nSuccessfully created edge types: [located_in].\nSuccessfully created reverse edge types: [reverse_located_in].\nSuccessfully created edge types: [associated_with].\nSuccessfully created reverse edge types: [reverse_associated_with].\nSuccessfully created edge types: [issued_card].\nSuccessfully created reverse edge types: [reverse_issued_card].\nSuccessfully created edge types: [ordered_from].\nSuccessfully created reverse edge types: [reverse_ordered_from].\nSuccessfully created edge types: [transacted_on].\nSuccessfully created reverse edge types: [reverse_transacted_on].'

In [11]:
conn.gsql(query_create_graph)

'The graph CzechGraph is created.'

After the queries are implemented, ensure with new connection.

In [43]:
# Define Graph
conn.graphname = graph_name

In [ ]:
# Create a secret
secret = conn.createSecret()
print(secret)

In [47]:
# Get auth Token
authToken = conn.getToken(secret)
print(authToken)

('4033ku76sqqf4300m7i29lalvbav1kkd', 1730289005, '2024-10-30 11:50:05')


/home/ekoteguh/Projects/IYKRA/GraphDB for Data Engineers/graphdb-tigergraph-study-case/venv/lib/python3.12/site-packages/pyTigerGraph/pyTigerGraphAuth.py:297: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  datetime.utcfromtimestamp(float(res.get("expiration"))).strftime('%Y-%m-%d %H:%M:%S')


In [48]:
# Re-connect
conn = tg.TigerGraphConnection(host=TG_HOST, graphname=graph_name, apiToken=authToken[0])

## Load Data

In [16]:
client_df = pd.read_csv("../datasets/client.csv", delimiter=",")
account_df = pd.read_csv("../datasets/account.csv", delimiter=",")
district_df = pd.read_csv("../datasets/district.csv", delimiter=",")
credit_card_df = pd.read_csv("../datasets/card.csv", delimiter=",")
loan_df = pd.read_csv("../datasets/loan.csv", delimiter=",")
permanent_order_df = pd.read_csv("../datasets/order.csv", delimiter=",")
disposition_df = pd.read_csv("../datasets/disp.csv", delimiter=",")
transaction_df = pd.read_csv("../datasets/trans.csv", delimiter=",", dtype=str)

Let's take a look at what inside the files looks like so we can write a loading job.

In [17]:
client_df.head()

,client_id,birth_number,district_id
0,1,706213,18
1,2,450204,1
2,3,406009,1
3,4,561201,5
4,5,605703,5


Here it's important to note that the `$0`, `$1` values line up with the columns of your data.
In this example:
- `$0` is the `id` column,
- `$1` is `birth_number`,
- `$2` is `district_id`
- and so on

### Creating Loading Jobs

#### Client

In [18]:
query_insert_client = """
USE GRAPH %s
BEGIN
CREATE LOADING JOB load_clients FOR GRAPH %s {
  DEFINE FILENAME CzechDataSource;
  LOAD CzechDataSource TO VERTEX Client VALUES($0, $1) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
}
END
""" % (graph_name, graph_name)

In [19]:
print(query_insert_client)


USE GRAPH CzechGraph
BEGIN
CREATE LOADING JOB load_clients FOR GRAPH CzechGraph {
  DEFINE FILENAME CzechDataSource;
  LOAD CzechDataSource TO VERTEX Client VALUES($0, $1) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
}
END



In [49]:
conn.gsql(query_insert_client)

HTTPError: 404 Client Error: Not Found for url: https://987c3d4375c043f5bf3691f630fdd055.i.tgcloud.io:443/restpp/gsql/file

#### Account

In [21]:
account_df.head()

,account_id,district_id,frequency,date
0,576,55,POPLATEK MESICNE,930101
1,3818,74,POPLATEK MESICNE,930101
2,704,55,POPLATEK MESICNE,930101
3,2378,16,POPLATEK MESICNE,930101
4,2632,24,POPLATEK MESICNE,930102


In [22]:
query_insert_account = """
USE GRAPH %s
BEGIN
CREATE LOADING JOB load_accounts FOR GRAPH %s {
  DEFINE FILENAME CzechDataSource;
  LOAD CzechDataSource TO VERTEX Client VALUES($0, $2, $3) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
}
END
""" % (graph_name, graph_name)

In [ ]:
# conn.gsql(query_insert_account)

#### District

In [23]:
district_df.head()

,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677.0,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159.0,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824.0,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244.0,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616.0,3040


We only take first two columns = `district_id` and `district_name`

In [24]:
query_insert_district = """
USE GRAPH %s
BEGIN
CREATE LOADING JOB load_districts FOR GRAPH %s {
  DEFINE FILENAME CzechDataSource;
  LOAD CzechDataSource TO VERTEX Client VALUES($0, $1) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
}
END
""" % (graph_name, graph_name)

In [ ]:
# conn.gsql(query_insert_district)

#### Credit Card

In [25]:
credit_card_df.head()

,card_id,disp_id,type,issued
0,1005,9285,classic,931107 00:00:00
1,104,588,classic,940119 00:00:00
2,747,4915,classic,940205 00:00:00
3,70,439,classic,940208 00:00:00
4,577,3687,classic,940215 00:00:00


In [26]:
query_insert_credit_card = """
USE GRAPH %s
BEGIN
CREATE LOADING JOB load_credit_cards FOR GRAPH %s {
  DEFINE FILENAME CzechDataSource;
  LOAD CzechDataSource TO VERTEX Client VALUES($0, $2, $3) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
}
END
""" % (graph_name, graph_name)

In [ ]:
# conn.gsql(query_insert_credit_card)

#### Loan

In [27]:
loan_df.head()

,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033.0,B
1,5316,1801,930711,165960,36,4610.0,A
2,6863,9188,930728,127080,60,2118.0,A
3,5325,1843,930803,105804,36,2939.0,A
4,7240,11013,930906,274740,60,4579.0,A


In [28]:
query_insert_loan = """
USE GRAPH %s
BEGIN
CREATE LOADING JOB load_loans FOR GRAPH %s {
  DEFINE FILENAME CzechDataSource;
  LOAD CzechDataSource TO VERTEX Client VALUES($0, $2, $3, $4, $5, $6) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
}
END
""" % (graph_name, graph_name)

In [ ]:
# conn.gsql(query_insert_loan)

#### Permanent Order

In [29]:
permanent_order_df.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,


In [30]:
query_insert_permanent_order = """
USE GRAPH %s
BEGIN
CREATE LOADING JOB load_permanent_orders FOR GRAPH %s {
  DEFINE FILENAME CzechDataSource;
  LOAD CzechDataSource TO VERTEX Client VALUES($0, $2, $3, $4, $5) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
}
END
""" % (graph_name, graph_name)

In [ ]:
# conn.gsql(query_insert_permanent_order)

#### Transaction

In [31]:
transaction_df.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,930101,PRIJEM,VKLAD,700.00,700.00,NaN,NaN,NaN
1,171812,576,930101,PRIJEM,VKLAD,900.00,900.00,NaN,NaN,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.00,1000.00,NaN,NaN,NaN
3,1117247,3818,930101,PRIJEM,VKLAD,600.00,600.00,NaN,NaN,NaN
4,579373,1972,930102,PRIJEM,VKLAD,400.00,400.00,NaN,NaN,NaN


In [32]:
query_insert_transaction = """
USE GRAPH %s
BEGIN
CREATE LOADING JOB load_transactions FOR GRAPH %s {
  DEFINE FILENAME CzechDataSource;
  LOAD CzechDataSource TO VERTEX Client VALUES($0, $2, $3, $4, $5, $6) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
}
END
""" % (graph_name, graph_name)

In [ ]:
# conn.gsql(query_insert_transaction)

#### Disposition

In [33]:
disposition_df.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT


In [34]:
query_insert_disposition = """
USE GRAPH %s
BEGIN
CREATE LOADING JOB load_dispositions FOR GRAPH %s {
  DEFINE FILENAME CzechDataSource;
  LOAD CzechDataSource TO VERTEX Client VALUES($0, $3) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
}
END
""" % (graph_name, graph_name)

In [ ]:
# conn.gsql(query_insert_disposition)

### Loading Data

In [35]:
client_file = "../datasets/client.csv"
account_file = "../datasets/account.csv"
district_file = "../datasets/district.csv"
credit_card_file = "../datasets/card.csv"
loan_file = "../datasets/loan.csv"
permanent_order_file = "../datasets/order.csv"
transaction_file = "../datasets/trans.csv"
disposition_file = "../datasets/disp.csv"

In [36]:
# Load the client file wiht the 'load_clients' job
client_result = conn.uploadFile(client_file, fileTag="CzechDataSource", jobName="load_clients")
print(json.dumps(client_result, indent=2))

/home/ekoteguh/Projects/IYKRA/GraphDB for Data Engineers/graphdb-tigergraph-study-case/venv/lib/python3.12/site-packages/pyTigerGraph/pyTigerGraphLoading.py:84: DeprecationWarning: The `uploadFile()` function is deprecated; use `runLoadingJobWithFile()` instead.
  warnings.warn(


TigerGraphException: ("Undefined loading job: load_clients for Graph 'CzechGraph'.", 'REST-30000')

In [ ]:
# Load the client file wiht the 'load_accounts' job
account_result = conn.uploadFile(account_file, fileTag="CzechDataSource", jobName="load_accounts")
print(json.dumps(account_result, indent=2))

In [ ]:
# Load the client file wiht the 'load_districts' job
district_result = conn.uploadFile(district_file, fileTag="CzechDataSource", jobName="load_districts")
print(json.dumps(district_result, indent=2))

In [ ]:
# Load the client file wiht the 'load_credit_cards' job
credit_card_result = conn.uploadFile(credit_card_file, fileTag="CzechDataSource", jobName="load_credit_cards")
print(json.dumps(credit_card_result, indent=2))

In [ ]:
# Load the client file wiht the 'load_loans' job
loan_result = conn.uploadFile(loan_file, fileTag="CzechDataSource", jobName="load_loans")
print(json.dumps(loan_result, indent=2))

In [ ]:
# Load the client file wiht the 'load_orders' job
permanent_order_result = conn.uploadFile(permanent_order_file, fileTag="CzechDataSource", jobName="load_permanent_orders")
print(json.dumps(permanent_order_result, indent=2))

In [ ]:
# Load the client file wiht the 'load_transactions' job
transaction_result = conn.uploadFile(transaction_file, fileTag="CzechDataSource", jobName="load_transactions")
print(json.dumps(transaction_result, indent=2))

In [ ]:
# Load the client file wiht the 'load_transactions' job
disposition_result = conn.uploadFile(disposition_file, fileTag="CzechDataSource", jobName="load_dispositions")
print(json.dumps(disposition_result, indent=2))

## Exploring the Graph

### Get Vertex and Edge Schema

In [ ]:
results = conn.getVertexTypes()
print(f"Verticies: {results}")
vertices = results

results = conn.getEdgeTypes()
print(f"Edges: {results}")
edges = results

### Counting Data

In [ ]:
print("Vertex Counts")
for vertex in vertices:
  print(f"There are {conn.getVertexCount(vertex)} {vertex} vertices in the graph")

print("--------------")
print("Edge Counts")
for edge in edges:
  print(f"There are {conn.getEdgeCount(edge)} {edge} edges in the graph")